<a href="https://colab.research.google.com/github/Arunsharma2004/Portfolio/blob/main/Exp_3_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical


In [4]:
# Load and preprocess MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = X_train.reshape((-1, 28, 28, 1)), X_test.reshape((-1, 28, 28, 1))


In [5]:
# Convert labels to one-hot encoding for classification
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)


In [6]:
# Define Autoencoder
input_layer = layers.Input(shape=(28, 28, 1))


In [7]:
# Encoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
encoded = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)


In [8]:
# Decoder
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

In [9]:
autoencoder = models.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
# Pretrain Autoencoder (Unsupervised)
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, shuffle=True)


Epoch 1/10
131/235 ━━━━━━━━━━━━━━━━━━━━ 1:14 714ms/step - loss: 0.2704

In [ ]:
# Extract Encoder for Greedy Pretraining
encoder = models.Model(input_layer, encoded)


In [ ]:
# Fine-Tuning (Supervised Learning)
encoded_input = layers.Input(shape=(14, 14, 64))
x = layers.Flatten()(encoded_input)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(10, activation='softmax')(x)


In [ ]:

classifier = models.Model(encoded_input, x)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Classifier with Encoded Features
classifier.fit(encoder.predict(X_train), y_train, epochs=5, batch_size=256, validation_data=(encoder.predict(X_test), y_test))

In [ ]:
import numpy as np

# Select some test images
n = 5
test_imgs = X_test[:n]
encoded_test_imgs = encoder.predict(test_imgs)

# Get classifier predictions
predictions = np.argmax(classifier.predict(encoded_test_imgs), axis=1)

# Plot results
plt.figure(figsize=(10, 2))
for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(test_imgs[i].reshape(28, 28), cmap='gray')
    plt.axis('off')
    plt.title(f"Pred: {predictions[i]}")
plt.show()

In [ ]:
\loss, accuracy = classifier.evaluate(encoder.predict(X_test), y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")